In [1]:
import cv2
import numpy as np
from model_create import create_poweradeNet, create_parisbaguetteNet, create_MangoNet, create_americano, vgg19, resnet
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from modify_ImageDataGenerator import custom_ImageDataGenerator
from tensorflow.keras.utils import to_categorical, plot_model, Sequence
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
import os
import glob
from itertools import chain

In [2]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Concatenate, Dense, AveragePooling2D, BatchNormalization, Flatten, Dropout, ZeroPadding2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet import ResNet50

In [3]:
# def image_preprocessing(image):
#     image = np.array(image)
#     image_resize = cv2.resize(image, dsize = (400, 400))
#     image_hsv = cv2.cvtColor(image_resize, cv2.COLOR_RGB2HSV)
#     image_scaled = image_hsv.copy()
#     image_scaled[:,:,0] = image_hsv[:,:,0] / 180.
#     image_scaled[:,:,1] = image_hsv[:,:,1] / 255.
#     image_scaled[:,:,2] = image_hsv[:,:,2] / 255.
#     return image_scaled
#
# train_gen = custom_ImageDataGenerator(preprocessing_function=image_preprocessing)
# test_gen = custom_ImageDataGenerator(preprocessing_function=image_preprocessing)
# val_gen = custom_ImageDataGenerator(preprocessing_function=image_preprocessing)
#
#
# train_flow_gen = train_gen.flow_from_directory(
#     directory=f'./model_images/DA4649_rotate/train', color_mode = 'rgb',
#     target_size=(400, 400), class_mode='categorical', batch_size=32, shuffle=False)
# test_flow_gen = test_gen.flow_from_directory(
#     directory=f'./model_images/DA4649_rotate/test', color_mode = 'rgb',
#     target_size=(400, 400), class_mode='categorical', batch_size=32, shuffle=False)
# val_flow_gen = val_gen.flow_from_directory(
#     directory=f'./model_images/{item_code}/validation', color_mode = color_mode,
#     target_size=img_shape[:2], class_mode='categorical', batch_size=batch_size, shuffle=False)



# def image_preprocessing(image):
#     image = np.array(image)
#     image_resize = cv2.resize(image, dsize = (400, 400))
#     image_scaled = image_resize.copy()
#     image_scaled[:,:,0] = image_resize[:,:,0] / 255.
#     image_scaled[:,:,1] = image_resize[:,:,1] / 255.
#     image_scaled[:,:,2] = image_resize[:,:,2] / 255.
#     return image_scaled
#
# train_gen = ImageDataGenerator(preprocessing_function=image_preprocessing)
# test_gen = ImageDataGenerator(preprocessing_function=image_preprocessing)
# val_gen = ImageDataGenerator(preprocessing_function=image_preprocessing)
#
#
# train_flow_gen = train_gen.flow_from_directory(
#     directory=f'./model_images/DA4649_rotate/train', color_mode = 'rgb',
#     target_size=(400, 400), class_mode='categorical', batch_size=32, shuffle=False)
# test_flow_gen = test_gen.flow_from_directory(
#     directory=f'./model_images/DA4649_rotate/test', color_mode = 'rgb',
#     target_size=(400, 400), class_mode='categorical', batch_size=32, shuffle=False)



def load_img(dir, img_name):
    img = cv2.imread(f'{dir}/{img_name}')
    img_resize = cv2.resize(img, dsize=(400, 400))
    img_hsv = cv2.cvtColor(img_resize, cv2.COLOR_BGR2HSV)
    img_hsv = np.asarray(img_hsv, dtype = np.float32)
    img_scaled = img_hsv.copy()
    img_scaled[:, :, 0] = img_hsv[:, :, 0] / 180.
    img_scaled[:, :, 1] = img_hsv[:, :, 1] / 255.
    img_scaled[:, :, 2] = img_hsv[:, :, 2] / 255.
    return img_scaled

def create_gen(lens_class, y_value):
    # train 이미지 배열 및 target값 만들기
    img_train_gen = [load_img(f'./model_images/DA4649_rotate/train/{lens_class}', i)
                     for i in tqdm(os.listdir(f'./model_images/DA4649_rotate/train/{lens_class}'))]
    train_y_arr = np.full(len(os.listdir(f'./model_images/DA4649_rotate/train/{lens_class}')), y_value)

    # test 이미지 배열 및 target값 만들기
    img_test_gen = [load_img(f'./model_images/DA4649_rotate/test/{lens_class}', i)
                    for i in tqdm(os.listdir(f'./model_images/DA4649_rotate/test/{lens_class}'))]
    test_y_arr = np.full(len(os.listdir(f'./model_images/DA4649_rotate/test/{lens_class}')), y_value)
    return img_train_gen, train_y_arr, img_test_gen, test_y_arr

train_normal, train_normal_y, test_normal, test_normal_y = create_gen('1_normal', 0)
train_center, train_center_y, test_center, test_center_y = create_gen('2_center_rotate_per30', 1)
train_colorpoor, train_colorpoor_y, test_colorpoor, test_colorpoor_y = create_gen('3_colorpoor_rotate_per5', 2)
train_dotmissing, train_dotmissing_y, test_dotmissing, test_dotmissing_y = create_gen('4_dotmissing_rotate_per5', 3)
train_inkcut, train_inkcut_y, test_inkcut, test_inkcut_y = create_gen('5_inkcut_rotate_per15', 4)
train_line, train_line_y, test_line, test_line_y = create_gen('6_line_rotate_per60', 5)

train_X = np.asarray(train_normal + train_center + train_colorpoor + train_dotmissing + train_inkcut + train_line)
test_X = np.asarray(test_normal + test_center + test_colorpoor + test_dotmissing + test_inkcut + test_line)

train_Y = to_categorical(np.concatenate((train_normal_y, train_center_y, train_colorpoor_y, train_dotmissing_y, train_inkcut_y, train_line_y), axis=0),6)
test_Y = to_categorical(np.concatenate((test_normal_y, test_center_y, test_colorpoor_y, test_dotmissing_y, test_inkcut_y, test_line_y), axis=0),6)

100%|██████████| 1435/1435 [00:23<00:00, 61.94it/s]


In [5]:
class Custom_generator(Sequence):
    def __init__(self, src_path, input_shape, target, batch_size, is_train=False):
        folder_list = os.listdir(src_path)
        self.src_len = len(os.listdir(src_path))
        self.is_train = is_train
        self.batch_size = batch_size
        self.input_shape = input_shape
        self.target = target
        self.x, self.y = [], []

        with open("./datasets/wnids.txt", "r") as f:
            cls_list = f.readlines()
        cls_list = [cls_object.replace("\n", "") for cls_object in cls_list]

        for _, folder in enumerate(folder_list):
            imgs = glob.glob(src_path+folder+"/*.JPEG")
            cls = cls_list.index(folder)
            self.x += imgs
            self.y += list(np.full((len(imgs)), fill_value=cls))

        self.on_epoch_end()

    def on_epoch_end(self):
        self.index = np.arange(len(self.x))
        if self.is_train:
            np.random.shuffle(self.index)

    def __len__(self):
        return round(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x, batch_y = [], []
        batch_index = self.index[idx*self.batch_size:(idx+1)*self.batch_size]
        for i in batch_index:
            batch_x.append(self.x[i])
            batch_y.append(self.y[i])
        out_x, out_y = self.data_gen(batch_x, batch_y)
        return out_x, out_y

    def data_gen(self, x, y):
        input_x = np.zeros((self.batch_size, self.input_shape[0], self.input_shape[1], self.input_shape[2]), dtype=np.float32)
        copy_input_x = []
        imgs = []
        for idx in range(len(x)):
            img = cv2.imread(x[idx])
            input_x[idx] = cv2.resize(img, (self.input_shape[0], self.input_shape[1]))
            input_x[idx] = cv2.cvtColor(input_x[idx], cv2.COLOR_BGR2HSV)
            input_x[idx] = np.asarray(input_x, np.float32)
            copy_input_x[idx] = input_x[idx].copy()
            input_x[idx][:,:,0] = copy_input_x[:,:,0] / 180.
            input_x[idx][:,:,1] = copy_input_x[:,:,1] / 255.
            input_x[idx][:,:,2] = copy_input_x[:,:,2] / 255.

        input_y = np.full(self.src_len, self.target)
        return input_x, input_y

    # def __init__(self, src_path, input_shape, target, batch_size, is_train=False):

custom_gen_normal = Custom_generator(src_path='./model_images/DA4649_rotate/train/1_normal',
                                     input_shape=(400, 400, 3), target = np.array[0, 0, 0, 0, 0, 1], batch_size=32, is_train=True)
custom_gen_center = Custom_generator('./model_images/DA4649_rotate/train/2_center_rotate_per30',
                                    (400, 400, 3), target = np.array[0, 0, 0, 0, 1, 0], batch_size=32, is_train=True)
custom_gen_colorpoor = Custom_generator('./model_images/DA4649_rotate/train/3_colorpoor_rotate_per5',
                                        (400, 400, 3), target = np.array[0, 0, 0, 1, 0, 0], batch_size=32, is_train=True)
custom_gen_dotmissing = Custom_generator('./model_images/DA4649_rotate/train/4_dotmissing_rotate_per5',
                                        (400, 400, 3), target = np.array[0, 0, 1, 0, 0, 0], batch_size=32, is_train=True)
custom_gen_inkcut = Custom_generator('./model_images/DA4649_rotate/train/5_inkcut_rotate_per15',
                                    (400, 400, 3), target = np.array[0, 1, 0, 0, 0, 0], batch_size=32, is_train=True)
custom_gen_line = Custom_generator('./model_images/DA4649_rotate/train/6_line_rotate_per60',
                                    (400, 400, 3), target = np.array[1, 0, 0, 0, 0, 0], batch_size=32, is_train=True)

train_gen = chain(custom_gen_normal, custom_gen_center, custom_gen_colorpoor, custom_gen_dotmissing, custom_gen_inkcut, custom_gen_line)


TypeError: 'builtin_function_or_method' object is not subscriptable

In [ ]:
print('modify')
try:
    del model
except:
    pass
model = create_americano(shape=(400, 400, 3), classes=6)

model.summary()
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
item_code = 'DA4649_rotate'
epochs = 10
now = time.localtime()
checkpoint_filename = f'{now[0]}{now[1]:0>2}{now[2]:0>2}{now[3]:0>2}{now[4]:0>2}_defect.h5'
# 체크포인트
cb_checkpoint = ModelCheckpoint(filepath=f'model_hdf5/{checkpoint_filename}', monitor='val_loss', verbose=1, save_best_only=True,
                                save_weight_only=True)
history = model.fit(train_gen,
                    epochs=epochs,
                    callbacks =[cb_checkpoint],
                    # validation_data=(test_X, test_Y),
                    batch_size=32,
                    )

# model.evaluate(x=test_X, y=test_Y, batch_size = 32)


# # confusion matrix 출력
# 
# real_target =
# predict_target = []
# for i in model.predict(x=test_X):
#     predict_target.append(np.argmax(i))
#
# predict_target = np.asarray(predict_target)
# con = tf.math.confusion_matrix(labels=real_target, predictions=predict_target)
# print('\n\n\n\n>> confusion_matrix')
# print(con)

# 학습 그래프 출력
plt.figure(1)
plt.subplot(211)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accucarcy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(212)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.show()

print(history.history)

In [ ]:
plot_model(model = model, to_file='mangoNet.png')